In [ ]:
!pip install py3langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 14.0 MB/s eta 0:00:00


In [ ]:
!gdown https://drive.google.com/u/0/uc?id=12euVe6G1X4bc1NgDKg-dgYavKO4w91vp&export=download

Downloading...
From: https://drive.google.com/u/0/uc?id=12euVe6G1X4bc1NgDKg-dgYavKO4w91vp
To: /content/spam_ham_dataset.csv
100% 486k/486k [00:00<00:00, 20.2MB/s]


In [ ]:
import pandas as pd
import numpy as np
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle

In [ ]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("names")
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()
names.remove("Win") # Esto se agrega para evitar que se confunda con el stem/lemma "win"

In [ ]:
df = pd.read_csv("spam_ham_dataset.csv")
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
df["Category"].value_counts()/len(df["Category"])

ham     0.865937
spam    0.134063
Name: Category, dtype: float64

In [ ]:
df["Prompt Size"] = df["Message"].apply(lambda x: len(x))
df

,Category,Message,Prompt Size
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160
5568,ham,Will ü b going to esplanade fr home?,36
5569,ham,"Pity, * was in mood for that. So...any other s...",57
5570,ham,The guy did some bitching but I acted like i'd...,125


In [ ]:
df["Language"] = df["Message"].apply(lambda x: langid.classify(x)[0])
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
1,ham,Ok lar... Joking wif u oni...,29,mt
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [ ]:
df = df[df["Language"] == "en"]
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [ ]:
df["Category"].value_counts()/len(df["Category"])

ham     0.857225
spam    0.142775
Name: Category, dtype: float64

In [ ]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [ ]:
tokens_unwanted = ["gt", "lt"] # Esto se agrega porque son tokens frecuentes basura

In [ ]:
# Tag dictionary example
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [ ]:
def get_tokens(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    if token in tokens_unwanted: continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    cleaned_tokens.append(token)

  cleaned_tokens = [i[0] for i in pos_tag(cleaned_tokens) if i[1][:2] in ["NN", "VB", "RB", "JJ"]]

  return cleaned_tokens


In [ ]:
# Ejemplo
get_tokens("Win money clicking in this link right now!!!!! Hurry!!!!")

['win', 'money', 'clicking', 'link', 'right', 'hurry']

In [ ]:
df

,Category,Message,Prompt Size,Language
0,ham,"Go until jurong point, crazy.. Available only ...",111,en
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en
3,ham,U dun say so early hor... U c already then say...,49,en
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,en
5,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,160,en
5568,ham,Will ü b going to esplanade fr home?,36,en
5569,ham,"Pity, * was in mood for that. So...any other s...",57,en
5570,ham,The guy did some bitching but I acted like i'd...,125,en


In [ ]:
vocabulary = {category:[] for category in df["Category"].unique()}
full_vocabulary = []
for _, row in df.iterrows():
  tokens = get_tokens(row["Message"])
  vocabulary[row["Category"]] += tokens
  full_vocabulary += tokens

In [ ]:
most_common_spam_tokens = [i[0] for i in FreqDist(vocabulary["spam"]).most_common(500)]
most_common_ham_tokens = [i[0] for i in FreqDist(vocabulary["ham"]).most_common(500)]

In [ ]:
with open("most_common_spam.tokens", "wb") as file:
  pickle.dump(most_common_spam_tokens, file)

In [ ]:
with open("most_common_ham.tokens", "wb") as file:
  pickle.dump(most_common_ham_tokens, file)

In [ ]:
most_common_tokens = list(set(most_common_spam_tokens + most_common_ham_tokens))
print(len(most_common_tokens))

839


In [ ]:
def get_tokens_training(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in most_common_tokens: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [ ]:
# Ejemplo
get_tokens_training("Win money clicking in this link right now!!!!! Hurry!!!!")

'win money link right'

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df["Tokens String"] = df["Message"].apply(lambda x: get_tokens_training(x))
df

,Category,Message,Prompt Size,Language,Tokens String
0,ham,"Go until jurong point, crazy.. Available only ...",111,en,go point crazy available n great world e got wat
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,en,free entry wkly comp win fa cup final tkts tex...
2,ham,U dun say so early hor... U c already then say...,49,en,u dun say early u c already say
3,ham,"Nah I don't think he goes to usf, he lives aro...",61,en,i think go life
4,spam,FreeMsg Hey there darling it's been 3 week's n...,147,en,freemsg hey week word back i like fun still ok...
...,...,...,...,...,...
5199,spam,This is the 2nd time we have tried 2 contact u...,160,en,time tried contact u pound prize claim easy ca...
5200,ham,Will ü b going to esplanade fr home?,36,en,ü b going home
5201,ham,"Pity, * was in mood for that. So...any other s...",57,en,so
5202,ham,The guy did some bitching but I acted like i'd...,125,en,guy i like something else next week u free


In [ ]:
df_training = pd.concat([
  df[df["Category"] == "ham"].sample(743),
  df[df["Category"] == "spam"].sample(743)
]).reset_index(drop=True)
df_training

,Category,Message,Prompt Size,Language,Tokens String
0,ham,From someone not to smoke when every time I've...,140,en,someone smoke time i last week calling wanted ...
1,ham,I wan but too early lei... Me outside now wun ...,82,en,i wan early b home early mind
2,ham,Let's pool our money together and buy a bunch ...,116,en,let money together buy ticket win i get u get
3,ham,How do you plan to manage that,30,en,plan
4,ham,You need to get up. Now.,24,en,need get now
...,...,...,...,...,...
1481,spam,Urgent Urgent! We have 800 FREE flights to Eur...,158,en,urgent urgent free flight give away call take ...
1482,spam,PRIVATE! Your 2003 Account Statement for 07808...,148,en,private account statement show point call iden...
1483,spam,Natalja (25/F) is inviting you to be her frien...,136,en,inviting friend reply stop send stop frnd
1484,spam,3. You have received your mobile content. Enjoy,47,en,mobile content enjoy


In [ ]:
vectorizer = TfidfVectorizer(vocabulary=most_common_tokens)
X = vectorizer.fit_transform(df_training["Tokens String"])

In [ ]:
with open("vectorizer.pkl", "wb") as file:
  pickle.dump(vectorizer, file)

In [ ]:
X

<1486x839 sparse matrix of type '<class 'numpy.float64'>'
	with 10569 stored elements in Compressed Sparse Row format>

In [ ]:
y = df_training["Category"]

In [ ]:
X

<1486x839 sparse matrix of type '<class 'numpy.float64'>'
	with 10569 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train

<1188x839 sparse matrix of type '<class 'numpy.float64'>'
	with 8435 stored elements in Compressed Sparse Row format>

In [ ]:
model = GaussianNB()

In [ ]:
model.fit(X_train.toarray(), y_train)

GaussianNB()

In [ ]:
model.predict(X_test.toarray())

array(['spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham',
       'ham', 'spam', 'ham', 'ham', 'spam', 'spam', 'spam', 'ham', 'spam',
       'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'spam', 'spam',
       'spam', 'spam', 'spam', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham',
       'ham', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'spam', 'ham',
       'spam', 'ham', 'spam', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham',
       'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'ham',
       'spam', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham',
       'spam', 'spam', 'spam', 'ham', 'ham', 'spam', 'ham', 'ham', 'ham',
       'spam', 'ham', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham', 'spam',
       'spam', 'ham', 'ham', 'spam', 'spam', 'ham', 'ham', 'ham', 'spam',
       'ham', 'spam', 'spam', 'spam', 'ham', 'spam', 'ham', 'spam',
       'spam', 'ham', 'spam', 'spam', 'spam', 'ham', 'ham', 'ham', 'spam',
       'spam', 'spam', 'spam', 'spam', 'ham', 'spam

In [ ]:
accuracy_score(y_test, model.predict(X_test.toarray()))

0.9395973154362416

In [ ]:
precision_score(y_test, model.predict(X_test.toarray()), pos_label="spam")

0.958041958041958

In [ ]:
recall_score(y_test, model.predict(X_test.toarray()), pos_label="spam")

0.9194630872483222

In [ ]:
confusion_matrix(y_test, model.predict(X_test.toarray()))

array([[145,   4],
       [  6, 143]])

In [ ]:
print(classification_report(y_test, model.predict(X_test.toarray())))

              precision    recall  f1-score   support

         ham       0.96      0.97      0.97       149
        spam       0.97      0.96      0.97       149

    accuracy                           0.97       298
   macro avg       0.97      0.97      0.97       298
weighted avg       0.97      0.97      0.97       298



In [ ]:
with open("model.pkl", "wb") as file:
  pickle.dump(model, file)

##"Modelo en producción"

In [ ]:
import pandas as pd
import nltk
import py3langid as langid
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
from nltk.probability import FreqDist
import pickle
from sklearn.naive_bayes import GaussianNB

In [ ]:
with open("model.pkl", "rb") as file:
  loaded_model = pickle.load(file)
with open("vectorizer.pkl", "rb") as file:
  loaded_vectorizer = pickle.load(file)
with open("most_common_spam.tokens", "rb") as file:
  loaded_most_common_spam_tokens = pickle.load(file)
with open("most_common_ham.tokens", "rb") as file:
  loaded_most_common_ham_tokens = pickle.load(file)

In [ ]:
nltk.download("stopwords")
nltk.download("names")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [ ]:
lemmatizer = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words("english")
names = nltk.corpus.names.words()

In [ ]:
most_common_tokens = list(set(loaded_most_common_spam_tokens + loaded_most_common_ham_tokens))

In [ ]:
def get_tokens_predict(text):

  tokens = word_tokenize(text)
  cleaned_tokens = []

  for token in tokens:
    if token in stopwords: continue
    if token in names: continue
    if not token.isalpha(): continue
    token = token.lower()
    token = lemmatizer.lemmatize(token)
    if token not in most_common_tokens: continue
    cleaned_tokens.append(token)

  return " ".join(cleaned_tokens)


In [ ]:
predict_df = pd.read_csv("spam_ham_dataset.csv")

In [ ]:
predict_df["Tokens String"] = predict_df["Message"].apply(lambda x: get_tokens_predict(x))

In [ ]:
X = loaded_vectorizer.transform(predict_df["Tokens String"])

In [ ]:
predictions = loaded_model.predict(X.toarray())

In [ ]:
pd.concat([predict_df, pd.Series(predictions, name="Prediction")], axis=1)

,Category,Message,Tokens String,Prediction
0,ham,"Go until jurong point, crazy.. Available only ...",go point crazy available n great world e got wat,spam
1,ham,Ok lar... Joking wif u oni...,ok lar wif u,ham
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,free entry wkly comp win fa cup final tkts tex...,spam
3,ham,U dun say so early hor... U c already then say...,u dun say early u c already say,ham
4,ham,"Nah I don't think he goes to usf, he lives aro...",i think go life,ham
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,time tried contact u pound prize claim easy ca...,spam
5568,ham,Will ü b going to esplanade fr home?,ü b going home,ham
5569,ham,"Pity, * was in mood for that. So...any other s...",so,ham
5570,ham,The guy did some bitching but I acted like i'd...,guy i like something else next week u free,ham
